In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


KeyboardInterrupt: ignored

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d frobert/handdrawn-shapes-hds-dataset

 97% 56.0M/57.7M [00:03<00:00, 22.1MB/s]
100% 57.7M/57.7M [00:03<00:00, 16.3MB/s]


In [ ]:
!unzip handdrawn-shapes-hds-dataset.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/user.u14/images/other/other.u14.0091.png  
  inflating: data/user.u14/images/other/other.u14.0092.png  
  inflating: data/user.u14/images/other/other.u14.0093.png  
  inflating: data/user.u14/images/other/other.u14.0094.png  
  inflating: data/user.u14/images/other/other.u14.0095.png  
  inflating: data/user.u14/images/other/other.u14.0096.png  
  inflating: data/user.u14/images/other/other.u14.0097.png  
  inflating: data/user.u14/images/other/other.u14.0098.png  
  inflating: data/user.u14/images/other/other.u14.0099.png  
  inflating: data/user.u14/images/other/other.u14.0100.png  
  inflating: data/user.u14/images/other/other.u14.0101.png  
  inflating: data/user.u14/images/other/other.u14.0102.png  
  inflating: data/user.u14/images/other/other.u14.0103.png  
  inflating: data/user.u14/images/other/other.u14.0104.png  
  inflating: data/user.u14/images/other/other.u14.0105.png  
  inflating: data/user.u14/images/other/other.u14

In [ ]:
import os
from distutils.dir_util import copy_tree

rootdir = './data'
folder_list = []
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        folder_list.append(d)

for temp in folder_list:
    for fname in ['ellipse','rectangle','triangle','other']:
        if os.path.isdir(temp+'/images/'+fname):
            copy_tree(temp+'/images/'+fname, './resource/'+fname)


In [ ]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
  './resource',
  color_mode='grayscale',
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode = 'categorical',
  image_size=(70, 70),
  batch_size=32)

val_ds = tf.keras.utils.image_dataset_from_directory(
  './resource',
  color_mode='grayscale',
  validation_split=0.2,
  label_mode = 'categorical',
  subset="validation",
  seed=123,
  image_size=(70, 70),
  batch_size=32)

Found 27292 files belonging to 4 classes.
Using 21834 files for training.
Found 27292 files belonging to 4 classes.
Using 5458 files for validation.


In [ ]:
train_ds.class_names

['ellipse', 'other', 'rectangle', 'triangle']

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch)
  break

(32, 70, 70, 1)
tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]], shape=(32, 4), dtype=float32)


In [ ]:
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Sequential


model = Sequential()
model.add(Flatten(input_shape=(70,70)))
model.add(Dense(4900*2,activation='relu'))
model.add(Dense(3000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(120,activation='relu'))
model.add(Dense(80,activation='relu'))
model.add(Dense(40,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 4900)              0         
                                                                 
 dense_12 (Dense)            (None, 9800)              48029800  
                                                                 
 dense_13 (Dense)            (None, 3000)              29403000  
                                                                 
 dense_14 (Dense)            (None, 1000)              3001000   
                                                                 
 dense_15 (Dense)            (None, 500)               500500    
                                                                 
 dense_16 (Dense)            (None, 120)               60120     
                                                                 
 dense_17 (Dense)            (None, 80)               

In [ ]:
image_batch, labels_batch = next(iter(train_ds))
labels_batch.shape

TensorShape([32, 4])

In [ ]:
model.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics=['acc'] )
model.fit(train_ds,epochs=10)

Epoch 1/10
683/683 [==============================] - 17s 20ms/step - loss: 53.7872 - acc: 0.3552
Epoch 2/10
683/683 [==============================] - 14s 20ms/step - loss: 1.1007 - acc: 0.5181
Epoch 3/10
683/683 [==============================] - 14s 20ms/step - loss: 0.9957 - acc: 0.5902
Epoch 4/10
683/683 [==============================] - 13s 20ms/step - loss: 0.9249 - acc: 0.6338
Epoch 5/10
683/683 [==============================] - 14s 21ms/step - loss: 0.8562 - acc: 0.6710
Epoch 6/10
683/683 [==============================] - 14s 20ms/step - loss: 0.9323 - acc: 0.6221
Epoch 7/10
683/683 [==============================] - 14s 20ms/step - loss: 1.2874 - acc: 0.3623
Epoch 8/10
683/683 [==============================] - 14s 20ms/step - loss: 1.3467 - acc: 0.3217
Epoch 9/10
430/683 [=================>............] - ETA: 5s - loss: 1.3866 - acc: 0.2599

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(val_ds)

171/171 [==============================] - 2s 8ms/step - loss: 1.3855 - acc: 0.2620


[1.3855183124542236, 0.2620007395744324]